In [1]:
import bitdotio 
from pprintpp import pprint

b = bitdotio.bitdotio(<password>)

conn = b.get_connection(<db>)
cur = conn.cursor()
cur.execute("SELECT 1")
pprint(cur.fetchone())

(1, )


In [2]:
import csv
from io import StringIO
import time

import pandas as pd
from sqlalchemy import create_engine
#from config import pg_string

In [3]:
# Create SQLAlchemy engine to manage our database connections
# Note that we bump the statement_timeout to 60 seconds
# pool_pre_ping=True 
engine = create_engine(<postgresql>
                       , pool_pre_ping=True)
# SQL for querying an entire table
sql = f'''
    SELECT *
    FROM "filled_final_table";
'''
# Return SQL query as a pandas dataframe
with engine.connect() as conn:
    # Set 1 minute statement timeout (units are milliseconds)
    conn.execute("SET statement_timeout = 60000;")
    df = pd.read_sql(sql, conn)
df.head()

,date,year,county,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,median_square_feet,average_listing_price,total_listing_count,violent_crime_rate,30_year_fixed_rate,number_of_schools,median_listing_price_mm,number_of_parks
0,2022-06,2022,Alamance,382500,298,18,452,12,108,2112,443362,644,None,5.52,NaN,0.034063,54.0
1,2022-06,2022,Alexander,312450,34,35,28,0,20,1942,370648,59,None,5.52,NaN,0.016428,25.0
2,2022-06,2022,Alleghany,389000,54,39,24,0,20,1904,422802,92,None,5.52,NaN,-0.038200,0.0
3,2022-06,2022,Anson,170000,22,34,16,0,8,1502,262240,48,None,5.52,NaN,0.218638,3.0
4,2022-06,2022,Ashe,485000,76,31,48,0,20,1907,592879,137,None,5.52,NaN,-0.088346,5.0


In [4]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [5]:
# change dtype object to float
df['violent_crime_rate']= df['violent_crime_rate'].str.replace(',','').astype(float)
df.head()

,date,year,county,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,median_square_feet,average_listing_price,total_listing_count,violent_crime_rate,30_year_fixed_rate,number_of_schools,median_listing_price_mm,number_of_parks
0,2022-06,2022,Alamance,382500,298,18,452,12,108,2112,443362,644,NaN,5.52,NaN,0.034063,54.0
1,2022-06,2022,Alexander,312450,34,35,28,0,20,1942,370648,59,NaN,5.52,NaN,0.016428,25.0
2,2022-06,2022,Alleghany,389000,54,39,24,0,20,1904,422802,92,NaN,5.52,NaN,-0.038200,0.0
3,2022-06,2022,Anson,170000,22,34,16,0,8,1502,262240,48,NaN,5.52,NaN,0.218638,3.0
4,2022-06,2022,Ashe,485000,76,31,48,0,20,1907,592879,137,NaN,5.52,NaN,-0.088346,5.0


In [6]:
df_new1 = df.drop(['date'], axis=1)
df_new1.dtypes

year                         int64
county                      object
median_listing_price         int64
active_listing_count         int64
median_days_on_market        int64
new_listing_count            int64
price_increased_count        int64
price_reduced_count          int64
median_square_feet           int64
average_listing_price        int64
total_listing_count          int64
violent_crime_rate         float64
30_year_fixed_rate         float64
number_of_schools          float64
median_listing_price_mm    float64
number_of_parks            float64
dtype: object

In [7]:
filter_df = df_new1[df_new1['year'] <2019]
#filter_df["Year"].unique()
filter_df.head()

,year,county,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,median_square_feet,average_listing_price,total_listing_count,violent_crime_rate,30_year_fixed_rate,number_of_schools,median_listing_price_mm,number_of_parks
4196,2018,Alamance,249900,761,78,172,28,116,2060,276731,843,419.9,4.64,94.0,0.021635,54.0
4197,2018,Alexander,229900,90,102,8,0,16,1924,258096,103,221.9,4.64,49.0,0.108219,25.0
4198,2018,Alleghany,114900,420,227,4,0,20,1886,179920,438,114.8,4.64,11.0,-0.000870,0.0
4199,2018,Anson,125000,65,100,8,0,12,1500,152823,78,568.4,4.64,15.0,-0.037721,3.0
4200,2018,Ashe,65000,899,214,28,0,52,1776,157405,957,NaN,4.64,7.0,0.000770,5.0


In [8]:

columns = [
     "year", "county",'median_listing_price',
    "active_listing_count", "median_days_on_market", "new_listing_count",
    "price_increased_count", "price_reduced_count", "median_square_feet",
    "total_listing_count", "violent_crime_rate","number_of_parks",
    "number_of_schools","30_year_fixed_rate"]

target = ["median_listing_price"]

In [9]:
# Load the data

df_data = filter_df.loc[:, columns].copy()
df_data = df_data.dropna()

df_data.reset_index(inplace=True, drop=True)

df_data.head()
df_data.isnull().sum()

year                     0
county                   0
median_listing_price     0
active_listing_count     0
median_days_on_market    0
new_listing_count        0
price_increased_count    0
price_reduced_count      0
median_square_feet       0
total_listing_count      0
violent_crime_rate       0
number_of_parks          0
number_of_schools        0
30_year_fixed_rate       0
dtype: int64

Split the Data into Training and Testing

In [41]:
# Create our features
X = df_data.drop('median_listing_price', axis=1)
from sklearn.preprocessing import OneHotEncoder

X = pd.get_dummies(X)

# Create our target
y = df_data.loc[:, target].copy()



In [42]:
data.head()

,year,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,median_square_feet,total_listing_count,violent_crime_rate,...,Union,Vance,Wake,Warren,Watauga,Wayne,Wilkes,Wilson,Yadkin,Yancey
0,2018,249900,761,78,172,28,116,2060,843,419.9,...,0,0,0,0,0,0,0,0,0,0
1,2018,229900,90,102,8,0,16,1924,103,221.9,...,0,0,0,0,0,0,0,0,0,0
2,2018,114900,420,227,4,0,20,1886,438,114.8,...,0,0,0,0,0,0,0,0,0,0
3,2018,125000,65,100,8,0,12,1500,78,568.4,...,0,0,0,0,0,0,0,0,0,0
4,2018,245000,272,151,32,0,48,1950,337,221.2,...,0,0,0,0,0,0,0,0,0,0


In [43]:
# Check the balance of our target values
y.value_counts()

median_listing_price
199900                  34
179900                  28
139900                  27
249900                  26
189900                  25
                        ..
243900                   1
244450                   1
244608                   1
244900                   1
74900                    1
Length: 1231, dtype: int64

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1) 
                                                 
X_test.shape


(653, 104)

In [45]:
#preprocess data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scale = scaler.transform(X_train)
X_test_scale = scaler.transform(X_test)

Balanced Random Forest Regressor

In [46]:
#https://scikit-learn.org/0.16/modules/generated/sklearn.ensemble.RandomForestRegressor.html
# Set up 
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=1) 
rf_model = rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test_scale)
Counter(y_train['median_listing_price'])

Counter({180000: 5,
         118950: 1,
         225000: 6,
         190623: 1,
         189700: 3,
         174950: 1,
         148000: 3,
         205000: 3,
         249950: 3,
         198250: 2,
         237500: 1,
         119990: 2,
         117500: 1,
         244500: 2,
         139900: 16,
         300000: 6,
         169900: 19,
         140000: 1,
         158000: 2,
         191450: 1,
         290000: 3,
         296450: 1,
         307566: 1,
         394000: 1,
         229900: 3,
         184900: 8,
         350000: 7,
         196000: 1,
         119900: 9,
         154900: 6,
         447500: 1,
         201724: 1,
         174900: 9,
         287900: 1,
         89995: 1,
         309587: 1,
         209900: 4,
         325000: 9,
         374900: 2,
         274900: 8,
         398900: 1,
         115000: 3,
         249900: 18,
         263000: 1,
         299900: 16,
         278700: 2,
         149500: 3,
         299000: 20,
         237000: 7,
         275000:

In [47]:
score = rf_model.score(X_train, y_train)
print(score) 

0.9954490523320548


In [48]:
score = rf_model.score(X_test, y_test)
print(f"Testing data score:{rf_model.score( X_test,y_test)}") 

Testing data score:0.9692113028682223


In [49]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)





[(0.5442003019136002, 'median_square_feet'),
 (0.09894084542959031, 'violent_crime_rate'),
 (0.06571638577469557, 'number_of_schools'),
 (0.04617886441577581, 'active_listing_count'),
 (0.044294241598976714, 'total_listing_count'),
 (0.028872891323431227, 'number_of_parks'),
 (0.014345163567894581, 'county_Dare'),
 (0.011183944091191686, 'county_Chatham'),
 (0.01112052538271959, 'price_reduced_count'),
 (0.010766806082427952, 'median_days_on_market'),
 (0.009652062079475318, 'county_Buncombe'),
 (0.00910441027915373, '30_year_fixed_rate'),
 (0.008548446992703594, 'price_increased_count'),
 (0.007118682389100697, 'new_listing_count'),
 (0.00547795058107623, 'county_Carteret'),
 (0.005450022679242218, 'county_Orange'),
 (0.005085098236647395, 'county_Davidson'),
 (0.004370427374740831, 'county_Polk'),
 (0.004158327080492539, 'county_Haywood'),
 (0.004047824915459341, 'county_Forsyth'),
 (0.0039685468248120675, 'county_Stanly'),
 (0.0038143145964262765, 'county_Clay'),
 (0.003455396696673

# Hypertune features

In [50]:
rf_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 1,
 'verbose': 0,
 'warm_start': False}

In [51]:
from sklearn.model_selection import RandomizedSearchCV

# Create the random grid search to use to find the best hyperparameters
random_grid = {'n_estimators': [100,200,300,400],
               'max_features': ['auto','sqrt', 'log2', None],
                'criterion':['squared_error','absolute_error','poisson'],}

rf = RandomForestRegressor()
# using 4 fold of cross validation, radom search parameters with 100 different 
# combinations. Use all avaiable cores.

rf_search = RandomizedSearchCV(rf, random_grid,
               n_iter = 100, cv =3, verbose = 2, random_state = 1)

# fit to model

rf_search.fit(X_train, y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] END criterion=squared_error, max_features=auto, n_estimators=100; total time=   2.6s
[CV] END criterion=squared_error, max_features=auto, n_estimators=100; total time=   1.9s
[CV] END criterion=squared_error, max_features=auto, n_estimators=100; total time=   2.9s
[CV] END criterion=squared_error, max_features=auto, n_estimators=200; total time=   9.7s
[CV] END criterion=squared_error, max_features=auto, n_estimators=200; total time=   6.7s
[CV] END criterion=squared_error, max_features=auto, n_estimators=200; total time=   3.6s
[CV] END criterion=squared_error, max_features=auto, n_estimators=300; total time=   4.9s
[CV] END criterion=squared_error, max_features=auto, n_estimators=300; total time=   5.0s
[CV] END criterion=squared_error, max_features=auto, n_estimators=300; total time=   4.8s
[CV] END criterion=squared_error, max_features=auto, n_estimators=400; total time=   6.0s
[CV] END criterion=squared_error, max_

[CV] END criterion=absolute_error, max_features=None, n_estimators=300; total time= 1.0min
[CV] END criterion=absolute_error, max_features=None, n_estimators=300; total time= 1.1min
[CV] END criterion=absolute_error, max_features=None, n_estimators=300; total time= 1.1min
[CV] END criterion=absolute_error, max_features=None, n_estimators=400; total time= 1.5min
[CV] END criterion=absolute_error, max_features=None, n_estimators=400; total time= 1.6min
[CV] END criterion=absolute_error, max_features=None, n_estimators=400; total time= 1.4min
[CV] END criterion=poisson, max_features=auto, n_estimators=100; total time=   1.8s
[CV] END criterion=poisson, max_features=auto, n_estimators=100; total time=   1.8s
[CV] END criterion=poisson, max_features=auto, n_estimators=100; total time=   1.8s
[CV] END criterion=poisson, max_features=auto, n_estimators=200; total time=   3.5s
[CV] END criterion=poisson, max_features=auto, n_estimators=200; total time=   3.6s
[CV] END criterion=poisson, max_fe

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   param_distributions={'criterion': ['squared_error',
                                                      'absolute_error',
                                                      'poisson'],
                                        'max_features': ['auto', 'sqrt', 'log2',
                                                         None],
                                        'n_estimators': [100, 200, 300, 400]},
                   random_state=1, verbose=2)

In [53]:
print(rf_search.best_params_)
print(rf_search.best_score_)

{'n_estimators': 200, 'max_features': 'sqrt', 'criterion': 'absolute_error'}
0.9587374257619629


## Run RandomForestRegressor with best parameters

In [60]:
from sklearn.ensemble import RandomForestRegressor
rf_model1 = RandomForestRegressor(n_estimators=200, max_features='sqrt', criterion= 'absolute_error',random_state=1) 
rf_model1 = rf_model.fit(X_train, y_train)
y_pred1 = rf_model.predict(X_test_scale)
Counter(y_train['median_listing_price'])

Counter({180000: 5,
         118950: 1,
         225000: 6,
         190623: 1,
         189700: 3,
         174950: 1,
         148000: 3,
         205000: 3,
         249950: 3,
         198250: 2,
         237500: 1,
         119990: 2,
         117500: 1,
         244500: 2,
         139900: 16,
         300000: 6,
         169900: 19,
         140000: 1,
         158000: 2,
         191450: 1,
         290000: 3,
         296450: 1,
         307566: 1,
         394000: 1,
         229900: 3,
         184900: 8,
         350000: 7,
         196000: 1,
         119900: 9,
         154900: 6,
         447500: 1,
         201724: 1,
         174900: 9,
         287900: 1,
         89995: 1,
         309587: 1,
         209900: 4,
         325000: 9,
         374900: 2,
         274900: 8,
         398900: 1,
         115000: 3,
         249900: 18,
         263000: 1,
         299900: 16,
         278700: 2,
         149500: 3,
         299000: 20,
         237000: 7,
         275000:

In [62]:
score1 = rf_model1.score(X_test, y_test)
print(f"Testing data score:{(score1)}") 

Testing data score:0.9692113028682223
